In [1]:
import pandas as pd
import pickle

In [2]:
%%time

individual_provider_rows =[]
for chunk in pd.read_csv('../data/Medicare_Provider_Utilization_and_Payment_Data__Physician_and_Other_Supplier_PUF_CY2017.csv', 
                         dtype={'Zip Code of the Provider': object},
                         chunksize = 50000):
    individual_provider_rows.append(chunk[chunk['Entity Type of the Provider'] == 'I']) 
               
                
individual_provider_payment_df = pd.concat(individual_provider_rows, ignore_index=True)

Wall time: 55.6 s


In [3]:
individual_provider_payment_df.shape

(9416125, 26)

In [4]:
individual_provider_payment_df.columns

Index(['National Provider Identifier',
       'Last Name/Organization Name of the Provider',
       'First Name of the Provider', 'Middle Initial of the Provider',
       'Credentials of the Provider', 'Gender of the Provider',
       'Entity Type of the Provider', 'Street Address 1 of the Provider',
       'Street Address 2 of the Provider', 'City of the Provider',
       'Zip Code of the Provider', 'State Code of the Provider',
       'Country Code of the Provider', 'Provider Type',
       'Medicare Participation Indicator', 'Place of Service', 'HCPCS Code',
       'HCPCS Description', 'HCPCS Drug Indicator', 'Number of Services',
       'Number of Medicare Beneficiaries',
       'Number of Distinct Medicare Beneficiary/Per Day Services',
       'Average Medicare Allowed Amount', 'Average Submitted Charge Amount',
       'Average Medicare Payment Amount',
       'Average Medicare Standardized Amount'],
      dtype='object')

In [5]:
individual_provider_payment_df_clean = individual_provider_payment_df.drop(columns = ['Middle Initial of the Provider', 
                                                                                      'Gender of the Provider',
                                                                                      'Street Address 2 of the Provider',
                                                                                      'Average Submitted Charge Amount',
                                                                                      'Average Medicare Standardized Amount'
                                                                                     ])


In [8]:
individual_provider_payment_df_clean.to_csv("../data/individual_provider_payment.csv", index = False)


In [7]:
individual_provider_payment_df_clean.to_pickle("../data/individual_provider_payment.pkl")